In [1]:
import base64
import json
import os
import random

import boto3

In [2]:
# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-east-1")
s3_client = boto3.client("s3")

In [3]:
# Set the model ID.
model_id = "amazon.titan-image-generator-v1"

In [17]:
# Get user input for the image generation prompt.
prompt = input("Enter your image description: ")

In [18]:
prompt


'Lion'

In [19]:
# Generate a random seed between 0 and 858,993,459
seed = random.randint(0, 858993460)

# Format the request payload using the model's native structure.
native_request = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {"text": prompt},
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 8.0,
        "height": 512,
        "width": 512,
        "seed": seed,
    },
}

In [20]:
# Convert the native request to JSON.
request = json.dumps(native_request)

In [21]:
# Invoke the model with the request.
response = client.invoke_model(
    modelId=model_id,
    body=request,
    contentType="application/json",
    accept="application/json"
)

In [22]:
# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract the image data.
base64_image_data = model_response["images"][0]

# Decode image data
image_data = base64.b64decode(base64_image_data)

In [23]:
# Define S3 bucket details
bucket_name = "jeet-store-generated-image"

In [24]:
image_filename = f"generated_images/titan_{seed}.png"

In [25]:
# Upload image directly to S3
s3_client.put_object(
    Bucket=bucket_name,
    Key=image_filename,
    Body=image_data,
    ContentType="image/png"  # Ensure the image displays in browser
)

{'ResponseMetadata': {'RequestId': '02FXKW56SAQ87JD1',
  'HostId': 'Qjn1NwO9O9noIYtuwxvIVv0VajtsXA+ay5ZDnFJYEQk1swQAn9vtRCB0CDC+SNPp7ZA0EcZ/rFw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Qjn1NwO9O9noIYtuwxvIVv0VajtsXA+ay5ZDnFJYEQk1swQAn9vtRCB0CDC+SNPp7ZA0EcZ/rFw=',
   'x-amz-request-id': '02FXKW56SAQ87JD1',
   'date': 'Thu, 06 Mar 2025 08:08:41 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"13c86743c4c1af5c1c02b34c987a30d7"',
   'x-amz-checksum-crc32': 'IIsnwg==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"13c86743c4c1af5c1c02b34c987a30d7"',
 'ChecksumCRC32': 'IIsnwg==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [26]:
# Generate the S3 URL
s3_url = f"https://{bucket_name}.s3.amazonaws.com/{image_filename}"
print(f"The generated image is available at: {s3_url}")

The generated image is available at: https://jeet-store-generated-image.s3.amazonaws.com/generated_images/titan_379035399.png
